$\newcommand{\giraffe}{\texttt{GiRaFFE}}$
# $\giraffe$: Solving GRFFE equations at a higher Finite Differencing order

### NRPy+ Source Code for this module: [Tutorial-GiRaFFE_Higher_Order.py](../edit/Tutorial-GiRaFFE_Higher_Order.py), which is fully documented in the [previous NRPy+ tutorial module](Tutorial-GiRaFFE_Higher_Order.ipynb) on using NRPy+ to construct Maxwell's equations and initial data as SymPy expressions.

This module focuses on using the equations developed in the [last tutorial](Tutorial-GiRaFFE_Higher_Order.ipynb) to build an Einstein Toolkit (ETK) thorn to solve the GRFFE equations in Cartesian coordinates. This tutorial will focus on implementing the time evolution aspects; others can be contructed to set up specific initial data.

When interfaced properly with the ETK, this module will propagate the initial data for $\tilde{S}_i$, $A_i$, and $\sqrt{\gamma} \Phi$, defined in the last tutorial, forward in time by integrating the equations for $\partial_t \tilde{S}_i$, $\partial_t A_i$ and $\partial_t [\sqrt{\gamma} \Phi]$ subject to spatial boundary conditions. The time evolution itself is handled by the $\text{MoL}$ (Method of Lines) thorn in the $\text{CactusNumerical}$ arrangement, and the boundary conditions by the $\text{Boundary}$ thorn in the $\text{CactusBase}$ arrangement. 

Similar to the other ETK modules we have built, we will construct the WaveToyNRPy module in two steps.

1. Call on NRPy+ to convert the SymPy expressions for the evolution equations into one C-code kernel.
1. Write the C code and linkages to the Einstein Toolkit infrastructure (i.e., the .ccl files) to complete this Einstein Toolkit module.


In [1]:
import NRPy_param_funcs as par
import indexedexp as ixp
import grid as gri
import finite_difference as fin
from outputC import *
import loop

#Step 0: Set the spatial dimension parameter to 3.
par.set_parval_from_str("grid::DIM", 3)
DIM = par.parval_from_str("grid::DIM")

# Step 1: Set the finite differencing order to 4.
par.set_parval_from_str("finite_difference::FD_CENTDERIVS_ORDER", 4)

# Step 1c: Call the MaxwellCartesian_Evol() function from within the
#          Maxwell/MaxwellCartesian_Evol.py module.
import Tutorial-GiRaFFE_Higher_Order as gho
gho.GiRaFFE_Higher_Order()

# Step 2: Register gridfunctions so they can be written to by NRPy.
#gammaDD = ixp.register_gridfunctions_for_single_rank2("AUX","gammaDD", "sym01",DIM=3)
#betaU   = ixp.register_gridfunctions_for_single_rank1("AUX","betaU",DIM=3)
#alpha   = gri.register_gridfunctions("AUX","alpha")
#ValenciavU = ixp.register_gridfunctions_for_single_rank1("AUX","ValenciavU",DIM=3)
Stilde_rhsD = ixp.register_gridfunctions_for_single_rank1("AUX","Stilde_rhs")
A_rhsD = ixp.register_gridfunctions_for_single_rank1("AUX","A_rhsD")
psi6Phi = gri.register_gridfunctions("AUX","psi6Phi")

# Step 3: Set the rhs gridfunctions to their variables 
#         defined by GiRaFFE_Higher_Order().


# Step 5: Create directories for the thorn if they don't exist.
!mkdir GiRaFFE_HO     2>/dev/null # 2>/dev/null: Don't throw an error if the directory already exists.
!mkdir GiRaFFE_HO/src 2>/dev/null # 2>/dev/null: Don't throw an error if the directory already exists.


SyntaxError: invalid syntax (<ipython-input-1-3d780ed8d0c7>, line 17)

We will also need a routine to compute new Valencia 3-velocities at each timestep using a conservative-to-primitive solver. Since we need $v^i_{(n)}$ everywhere, this will require us to compute $B^i$ everywhere. However, $B^i = \epsilon^{ijk} \partial_j A_k$ requires derivatives of $A_i$, so getting $B^i$ will require some finesse. A chief aspect of this will require using lower-order finite differencing in the ghost zones. To that end, we will create .h files for each finite differencing order $\leq 10$, as well as upwinded- and downwinded-derivatives at 2nd order. These will let us compute the derivative at the outermost gridpoints.

In [ ]:
AD = ixp.register_gridfunctions_for_single_rank1("AUX","AD",DIM=3)
# Step 2: Import the four metric
gammaUU = ixp.register_gridfunctions_for_single_rank2("AUX","gammaUU","sym01")
gammaDET = gri.register_gridfunctions("AUX","gammaDET")
gammaUU, gammaDET = ixp.symm_matrix_inverter3x3(gammaDD)

# We already have a handy function to define the Levi-Civita symbol in WeylScalars
import WeylScal4NRPy.WeylScalars_Cartesian as weyl
LeviCivitaDDD = weyl.define_LeviCivitaSymbol_rank3()
LeviCivitaUUU = ixp.zerorank3()
for i in range(DIM):
    for j in range(DIM):
        for k in range(DIM):
            LCijk = LeviCivitaDDD[i][j][k]
            LeviCivitaDDD[i][j][k] = LCijk * sp.sqrt(gammaDET)
            LeviCivitaUUU[i][j][k] = LCijk / sp.sqrt(gammaDET)

AD_dD = ixp.declarerank2("AD_dD","nosym")
BU = ixp.register_gridfunctions_for_single_rank1("AUX","BU")
for i in range(DIM):
    BU[i] = 0
    for j in range(DIM):
        for k in range(DIM):
            BU[i] += LeviCivitaUUU[i][j][k] * AD_dD[k][j]

par.set_parval_from_str("finite_difference::FD_CENTDERIVS_ORDER", 10)
BU_to_print = fin.FD_outputC("GiRaFFE_HO/src/B_from_A_10.h",[\
                                                         lhrh(lhs=gri.gfaccess("out_gfs","BU0"),rhs=BU[0]),\
                                                         lhrh(lhs=gri.gfaccess("out_gfs","BU1"),rhs=BU[1]),\
                                                         lhrh(lhs=gri.gfaccess("out_gfs","BU2"),rhs=BU[2])])

par.set_parval_from_str("finite_difference::FD_CENTDERIVS_ORDER", 8)
BU_to_print = fin.FD_outputC("GiRaFFE_HO/src/B_from_A_8.h",[\
                                                        lhrh(lhs=gri.gfaccess("out_gfs","BU0"),rhs=BU[0]),\
                                                        lhrh(lhs=gri.gfaccess("out_gfs","BU1"),rhs=BU[1]),\
                                                        lhrh(lhs=gri.gfaccess("out_gfs","BU2"),rhs=BU[2])])

par.set_parval_from_str("finite_difference::FD_CENTDERIVS_ORDER", 6)
BU_to_print = fin.FD_outputC("GiRaFFE_HO/src/B_from_A_6.h",[\
                                           lhrh(lhs=gri.gfaccess("out_gfs","BU0"),rhs=BU[0]),\
                                           lhrh(lhs=gri.gfaccess("out_gfs","BU1"),rhs=BU[1]),\
                                           lhrh(lhs=gri.gfaccess("out_gfs","BU2"),rhs=BU[2])])

par.set_parval_from_str("finite_difference::FD_CENTDERIVS_ORDER", 4)
BU_to_print = fin.FD_outputC("GiRaFFE_HO/src/B_from_A_4.h",[\
                                                        lhrh(lhs=gri.gfaccess("out_gfs","BU0"),rhs=BU[0]),\
                                                        lhrh(lhs=gri.gfaccess("out_gfs","BU1"),rhs=BU[1]),\
                                                        lhrh(lhs=gri.gfaccess("out_gfs","BU2"),rhs=BU[2])])

par.set_parval_from_str("finite_difference::FD_CENTDERIVS_ORDER", 2)
BU_to_print = fin.FD_outputC("GiRaFFE_HO/src/B_from_A_2.h",[\
                                                        lhrh(lhs=gri.gfaccess("out_gfs","BU0"),rhs=BU[0]),\
                                                        lhrh(lhs=gri.gfaccess("out_gfs","BU1"),rhs=BU[1]),\
                                                        lhrh(lhs=gri.gfaccess("out_gfs","BU2"),rhs=BU[2])])

AD_ddnD = ixp.declarerank2("AD_ddnD","nosym")
for i in range(DIM):
    BU[i] = 0
    for j in range(DIM):
        for k in range(DIM):
            BU[i] += LeviCivitaUUU[i][j][k] * AD_ddnD[k][j]
BU_to_print = fin.FD_outputC("GiRaFFE_HO/src/B_from_A_D.h",[\
                                                        lhrh(lhs=gri.gfaccess("out_gfs","BU0"),rhs=BU[0]),\
                                                        lhrh(lhs=gri.gfaccess("out_gfs","BU1"),rhs=BU[1]),\
                                                        lhrh(lhs=gri.gfaccess("out_gfs","BU2"),rhs=BU[2])])

AD_dupD = ixp.declarerank2("AD_dupD","nosym")
for i in range(DIM):
    BU[i] = 0
    for j in range(DIM):
        for k in range(DIM):
            BU[i] += LeviCivitaUUU[i][j][k] * AD_dupD[k][j]
BU_to_print = fin.FD_outputC("GiRaFFE_HO/src/B_from_A_U.h",[\
                                                        lhrh(lhs=gri.gfaccess("out_gfs","BU0"),rhs=BU[0]),\
                                                        lhrh(lhs=gri.gfaccess("out_gfs","BU1"),rhs=BU[1]),\
                                                        lhrh(lhs=gri.gfaccess("out_gfs","BU2"),rhs=BU[2])])



### Step 2: Interfacing with the Einstein Toolkit

#### Step 2a: Constructing the Einstein Toolkit C-code calling functions that include the C code kernels.

Now that we have generated the C code kernel *GiRaFFE_RHSs.h* and the parameters file *NRPy_params.h*, we will need to write C code to make use of these files. To do this, we can simply follow the example within the [IDScalarWaveNRPy tutorial module](Tutorial-ETK_thorn-IDScalarWaveNRPy.ipynb). Functions defined by these files will be called by the Einstein Toolkit scheduler (specified in schedule.ccl below).

Also, here we will write the logic that determines which files are called where in order to calculate $B^i$. 
1. Take the primary finite differencing order $N$ from the param.ccl file. Fill in the interior points with the corresponding FD order. 
1. Then, at each face, at $0+\text{cctk_nghostzones[face]}-1$ and $\text{cctk_lsh[face]}-\text{cctk_nghostzones[face]}+1$, calculate B at order $N-2$
1. Continue moving outwards: at the points $0+\text{cctk_nghostzones[face]}-p$ and $\text{cctk_lsh[face]}-\text{cctk_nghostzones[face]}+p$, calculate B at order $N-2p$.
1. When $\text{cctk_nghostzones[face]}-p = 0$, use the upwinding derivatives 

In [ ]:
%%writefile GiRaFFE_HO/src/MaxwellEvol.c
#include <math.h>
#include <stdio.h>

#include "cctk.h"
#include "cctk_Arguments.h"
#include "cctk_Parameters.h"
#include "Symmetry.h"

#include "NRPy_params.h"

const int MAXFACE = -1;
const int NUL     = +0;
const int MINFACE = +1;


// Declare boundary condition FACE_UPDATE function,
// which fills in the ghost zones with successively
// lower order finite differencing
void AtoB(const int ORDER, 
          const CCTK_REAL *Ax,const CCTK_REAL *Ay,const CCTK_REAL *Az,
          const CCTK_REAL *Bx,const CCTK_REAL *By,const CCTK_REAL *Bz,
          const int i0min, const int i0max, 
          const int i1min, const int i1max, 
          const int i2min, const int i2max, 
          const int FACEX0, const int FACEX1, const int FACEX2) {
  if(ORDER==8) {
    for(int i2=i2min;i2<i2max;i2++) for(int i1=i1min;i1<i1max;i1++) for(int i0=i0min;i0<i0max;i0++) {
        #include "B_from_A_8.h"
    }
  } else if(ORDER==6) {
    for(int i2=i2min;i2<i2max;i2++) for(int i1=i1min;i1<i1max;i1++) for(int i0=i0min;i0<i0max;i0++) {
        #include "B_from_A_6.h"
    }
  } else if(ORDER==4) {
    for(int i2=i2min;i2<i2max;i2++) for(int i1=i1min;i1<i1max;i1++) for(int i0=i0min;i0<i0max;i0++) {
        #include "B_from_A_4.h"
    }
  } else if(ORDER==2) {
    for(int i2=i2min;i2<i2max;i2++) for(int i1=i1min;i1<i1max;i1++) for(int i0=i0min;i0<i0max;i0++) {
        #include "B_from_A_2.h"
    } 
  } else if(ORDER==0) {
    if(FACEX0==MAXFACE) {
        for(int i2=i2min;i2<i2max;i2++) for(int i1=i1min;i1<i1max;i1++) for(int i0=i0min;i0<i0max;i0++) {
            #include "B_from_A_2x0D.h"
        }
    } else if(FACEX0==MINFACE) {
        for(int i2=i2min;i2<i2max;i2++) for(int i1=i1min;i1<i1max;i1++) for(int i0=i0min;i0<i0max;i0++) {
            #include "B_from_A_2x0U.h"
        }
    } else if(FACEX1==MAXFACE) {
        for(int i2=i2min;i2<i2max;i2++) for(int i1=i1min;i1<i1max;i1++) for(int i0=i0min;i0<i0max;i0++) {
            #include "B_from_A_2x1D.h"
        }
    } else if(FACEX1==MINFACE) {
        for(int i2=i2min;i2<i2max;i2++) for(int i1=i1min;i1<i1max;i1++) for(int i0=i0min;i0<i0max;i0++) {
            #include "B_from_A_2x1U.h"
        }
    } else if(FACEX2==MAXFACE) {
        for(int i2=i2min;i2<i2max;i2++) for(int i1=i1min;i1<i1max;i1++) for(int i0=i0min;i0<i0max;i0++) {
            #include "B_from_A_2x2D.h"
        }
    } else if(FACEX2==MINFACE) {
        for(int i2=i2min;i2<i2max;i2++) for(int i1=i1min;i1<i1max;i1++) for(int i0=i0min;i0<i0max;i0++) {
            #include "B_from_A_2x2U.h"
        }
    } else {
        printf("ERROR. FACEX parameters not set properly.\n")
        exit(1);
    }
  } else {
    printf("ERROR. ORDER parameter not set properly.\n")
    exit(1);
  }
}

void driver_A_to_B(CCTK_ARGUMENTS) {
  const int *NG = cctk_nghostzones;
  const int *Nx = cctk_lsh;
  CCTK_INT ORDER = NG[0]*2;
  AtoB(ORDER, Ax,Ay,Az,Bx,By,Bz, NG[0],Nx[0]-NG[0],NG[1],Nx[1]-NG[1],NG[2],Nx[2]-NG[2]);
  int imin[3] = { NG[0], NG[1], NG[2] };
  int imax[3] = { Nx[0]-NG[0], Nx[1]-NG[1], Nx[2]-NG[2] };
  for(int which_gz = 0; which_gz < NG[0]; which_gz++) {
      // After updating each face, adjust imin[] and imax[] 
      //   to reflect the newly-updated face extents.
      ORDER -= which_gz*2;
      AtoB(ORDER, Ax,Ay,Az,Bx,By,Bz, imin[0]-1,imin[0], imin[1],imax[1], imin[2],imax[2], MINFACE,NUL,NUL); imin[0]--;
      AtoB(ORDER, Ax,Ay,Az,Bx,By,Bz, imax[0],imax[0]+1, imin[1],imax[1], imin[2],imax[2], MAXFACE,NUL,NUL); imax[0]++;

      AtoB(ORDER, Ax,Ay,Az,Bx,By,Bz, imin[0],imax[0], imin[1]-1,imin[1], imin[2],imax[2], NUL,MINFACE,NUL); imin[1]--;
      AtoB(ORDER, Ax,Ay,Az,Bx,By,Bz, imin[0],imax[0], imax[1],imax[1]+1, imin[2],imax[2], NUL,MAXFACE,NUL); imax[1]++;

      AtoB(ORDER, Ax,Ay,Az,Bx,By,Bz, imin[0],imax[0], imin[1],imax[1], imin[2]-1,imin[2], NUL,NUL,MINFACE); imin[2]--;
      AtoB(ORDER, Ax,Ay,Az,Bx,By,Bz, imin[0],imax[0], imin[1],imax[1], imax[2],imax[2]+1, NUL,NUL,MAXFACE); imax[2]++;
    }
}

#### Step 2b: CCL files - Define how this module interacts and interfaces with the larger Einstein Toolkit infrastructure

Writing a module ("thorn") within the Einstein Toolkit requires that three "ccl" files be constructed, all in the root directory of the thorn:

1. $\text{interface.ccl}$: defines the gridfunction groups needed, and provides keywords denoting what this thorn provides and what it should inherit from other thorns. This file governs the interaction between this thorn and others; more information can be found in the [official Einstein Toolkit documentation](http://cactuscode.org/documentation/referencemanual/ReferenceManualch8.html#x12-260000C2.2). 
With "implements", we give our thorn its unique name. By "inheriting" other thorns, we tell the Toolkit that we will rely on variables that exist within those functions. Then, we tell the toolkit that we want the gridfunctions $A_i$, $\tilde{S}_i$, and $\sqrt{\gamma}\Phi$ to be visible to other thorns by using the keyword "public". 

In [ ]:
%%writefile GiRaFFE_HO/interface.ccl

2. $\text{param.ccl}$: specifies free parameters within the thorn, enabling them to be set at runtime. It is required to provide allowed ranges and default values for each parameter. More information on this file's syntax can be found in the [official Einstein Toolkit documentation](http://cactuscode.org/documentation/referencemanual/ReferenceManualch8.html#x12-265000C2.3). A number of parameters are defined, and more parameters can be easily added in later versions. We also set the number of timelevels we will store in memory.

In [ ]:
%%writefile GiRaFFE_HO/param.ccl


3. $\text{schedule.ccl}$: allocates storage for gridfunctions, defines how the thorn's functions should be scheduled in a broader simulation, and specifies the regions of memory written to or read from gridfunctions. $\text{schedule.ccl}$'s official documentation may be found [here](http://cactuscode.org/documentation/referencemanual/ReferenceManualch8.html#x12-268000C2.4). 

We first assign storage for both scalar gridfunctions, and then specify the standardized ETK "scheduling bins" in which we want each of our thorn's functions to run.

In [ ]:
%%writefile GiRaFFE_HO/schedule.ccl


#### Step 2c: Add the C file to Einstein Toolkit compilation list.

We will also need $\text{make.code.defn}$, which indicates the list of files that need to be compiled. This thorn only has the one C file to compile.

In [ ]:
%%writefile GiRaFFE_HO/src/make.code.defn
